In [1]:
# CELLA 1: SETUP
import os
import sys

try:
    import ultralytics
except ImportError:
    !pip install ultralytics
    import ultralytics
import cv2
import numpy as np
from google.colab import files
import json
from collections import deque

print("✅ Ambiente pronto. Carica i tuoi video ORIGINALI (video1.mp4, ecc).")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Ambiente pronto. Carica i tuoi video ORIGINALI (video1.mp4, ecc).


In [5]:
# =============================================================================
# CELL 2: THE TRACKER "IL SEGUGIO" (FINAL VERSION: STABILIZED)
# =============================================================================
class IlSegugio:
    def __init__(self, model_ver='yolov8x.pt'):
        print(f"Initializing 'Il Segugio' (Stabilized & Enhanced)...")
        try:
            # If you have a specific tennis model, use it here. Otherwise, X or M.
            self.model = YOLO(model_ver)
        except:
            print("Model not found, using yolov8m.pt as backup.")
            self.model = YOLO('yolov8m.pt')
        self.trajectory = []

        # MOTION DETECTOR (OPTIMIZED)
        # history=20: Adapts quickly to light changes
        # varThreshold=16: High sensitivity for fast, faint balls
        self.back_sub = cv2.createBackgroundSubtractorMOG2(history=20, varThreshold=16, detectShadows=False)
        self.avg_area = None

    def applica_clahe(self, frame):
        # Local Contrast Enhancement
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        limg = cv2.merge((cl, a, b))
        return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    def check_colore_palla(self, roi_img):
        """
        Checks for Tennis Yellow OR Bright White (Sunlit ball).
        """
        if roi_img.size == 0: return 0

        # Convert to HSV
        hsv = cv2.cvtColor(roi_img, cv2.COLOR_BGR2HSV)

        # 1. Tennis Yellow Range
        lower_yellow = np.array([25, 30, 60])
        upper_yellow = np.array([50, 255, 255])
        mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

        # 2. Bright White Range (For overexposed fast balls)
        # Low Saturation, High Value
        lower_white = np.array([0, 0, 180])
        upper_white = np.array([180, 50, 255])
        mask_white = cv2.inRange(hsv, lower_white, upper_white)

        # Combine masks
        final_mask = cv2.bitwise_or(mask_yellow, mask_white)

        # Calculate ratio
        ratio = cv2.countNonZero(final_mask) / (roi_img.shape[0] * roi_img.shape[1])
        return ratio

    def pulisci_traiettoria(self, data):
        # Interpolates missing points if gaps are small
        if len(data) < 3: return data
        cleaned = [data[0]]
        MAX_JUMP = 250

        for i in range(1, len(data) - 1):
            prev = data[i-1]
            curr = data[i]
            next_p = data[i+1]

            d1 = np.sqrt((curr['u'] - prev['u'])**2 + (curr['v'] - prev['v'])**2)
            d2 = np.sqrt((curr['u'] - next_p['u'])**2 + (curr['v'] - next_p['v'])**2)
            d3 = np.sqrt((prev['u'] - next_p['u'])**2 + (prev['v'] - next_p['v'])**2)

            if d1 > MAX_JUMP and d2 > MAX_JUMP and d3 < MAX_JUMP:
                fixed = curr.copy()
                fixed['u'] = (prev['u'] + next_p['u']) / 2
                fixed['v'] = (prev['v'] + next_p['v']) / 2
                fixed['type'] = 'fixed'
                cleaned.append(fixed)
            else:
                cleaned.append(curr)
        cleaned.append(data[-1])
        return cleaned

    def processa_video(self, video_input, target_w_export, target_h_export, video_output=None):
        cap = cv2.VideoCapture(video_input)
        if not cap.isOpened():
            print(f"❌ Error opening: {video_input}")
            return []

        orig_w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        orig_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps    = cap.get(cv2.CAP_PROP_FPS)

        scale_x = target_w_export / orig_w
        scale_y = target_h_export / orig_h

        print(f"▶️ Processing: {video_input} ({orig_w}x{orig_h})")

        out = None
        if video_output:
            out = cv2.VideoWriter(video_output, cv2.VideoWriter_fourcc(*'mp4v'), fps, (orig_w, orig_h))

        frame_id = 0
        self.trajectory = []
        self.avg_area = None

        last_u, last_v = None, None
        velocity_u, velocity_v = 0, 0
        missed_frames = 0

        # === TUNED PARAMETERS ===
        MAX_MISSING = 12       # Reduced to stop long bad predictions
        SEARCH_RADIUS = 900
        CONF_THRESHOLD = 0.01
        FRICTION = 0.90        # Increased friction to prevent "flying away"
        GRAVITY = 0.75
        RESET_AFTER = 6

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break

            # 1. MOTION MASK
            motion_mask = self.back_sub.apply(frame)
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
            motion_mask = cv2.dilate(motion_mask, kernel, iterations=2)

            # 2. YOLO DETECTION
            frame_enhanced = self.applica_clahe(frame)
            # imgsz=1280 is faster and often sufficient for tennis
            results = self.model(frame_enhanced, verbose=False, classes=[32], conf=CONF_THRESHOLD, imgsz=1280)

            best_candidate = None
            best_candidate_area = 0
            min_dist = float('inf')

            if missed_frames > RESET_AFTER:
                last_u, self.avg_area = None, None

            current_radius = SEARCH_RADIUS + (missed_frames * 200)

            for result in results:
                for box in result.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    area = (x2-x1) * (y2-y1)
                    u, v = (x1 + x2) / 2, (y1 + y2) / 2
                    conf = float(box.conf)

                    # --- ASPECT RATIO FILTER (Handling Motion Blur) ---
                    w_box = x2 - x1
                    h_box = y2 - y1
                    if h_box > 0:
                        ar = w_box / h_box
                        # Reject extremely long lines or vertical poles
                        # Tennis balls at speed can be flattened (ar > 1)
                        if ar > 5.0 or ar < 0.2: continue

                    # A. EDGE FILTER (Ignore crowd/sky)
                    if y2 < orig_h * 0.15: continue

                    # B. MOTION FILTER
                    y1_int, x1_int = max(0, int(y1)), max(0, int(x1))
                    y2_int, x2_int = min(orig_h, int(y2)), min(orig_w, int(x2))
                    roi_motion = motion_mask[y1_int:y2_int, x1_int:x2_int]

                    motion_ratio = 0
                    if roi_motion.size > 0:
                        motion_ratio = cv2.countNonZero(roi_motion) / roi_motion.size
                        if motion_ratio < 0.05: continue

                    # C. AREA FILTER
                    if area < 5 or area > 8000: continue
                    if self.avg_area is not None:
                        ratio_area = area / self.avg_area
                        # Allow larger size changes if ball comes towards camera
                        if ratio_area > 8.0 or ratio_area < 0.10: continue

                    # D. COLOR FILTER
                    roi_color = frame[y1_int:y2_int, x1_int:x2_int]
                    yellow_score = self.check_colore_palla(roi_color)

                    is_yellow = yellow_score > 0.05

                    dist = float('inf')
                    if last_u is not None:
                        dist = np.sqrt((u - last_u)**2 + (v - last_v)**2)

                    # DECISION LOGIC
                    is_compatible = False

                    if last_u is not None and dist < current_radius:
                        is_compatible = True
                    elif conf > 0.15:
                        # If YOLO is confident, check it's not static noise
                        if is_yellow or motion_ratio > 0.2:
                            is_compatible = True
                    elif last_u is None and (conf > 0.1):
                        is_compatible = True

                    if is_compatible:
                        # Score: Prioritize Proximity, then Color
                        score = (10000 - dist) + (yellow_score * 5000)

                        if last_u is not None:
                             if dist < min_dist:
                                min_dist = dist
                                best_candidate = (u, v)
                                best_candidate_area = area
                        else:
                             if best_candidate is None or (yellow_score > 0 and conf > 0.1):
                                best_candidate = (u, v)
                                best_candidate_area = area

            final_u, final_v = None, None

            if best_candidate:
                curr_u, curr_v = best_candidate
                final_u, final_v = curr_u, curr_v

                if self.avg_area is None: self.avg_area = best_candidate_area
                else: self.avg_area = 0.90 * self.avg_area + 0.10 * best_candidate_area

                # === VELOCITY SMOOTHING (FIX FOR RED BALL JITTER) ===
                if last_u is not None:
                    inst_vel_u = curr_u - last_u
                    inst_vel_v = curr_v - last_v

                    # Sanity Check: Ignore impossible jumps (>150px)
                    if abs(inst_vel_u) < 150 and abs(inst_vel_v) < 150:
                        if velocity_u == 0 and velocity_v == 0:
                            velocity_u = inst_vel_u
                            velocity_v = inst_vel_v
                        else:
                            # Weighted Average: 30% New, 70% Inertia
                            velocity_u = 0.3 * inst_vel_u + 0.7 * velocity_u
                            velocity_v = 0.3 * inst_vel_v + 0.7 * velocity_v
                # ====================================================

                last_u, last_v = curr_u, curr_v
                missed_frames = 0

            elif last_u is not None and missed_frames < MAX_MISSING:
                # Prediction Mode
                velocity_u *= FRICTION
                velocity_v *= FRICTION
                velocity_v += GRAVITY
                pred_u = last_u + velocity_u
                pred_v = last_v + velocity_v

                # Bounds check
                if 0 <= pred_u <= orig_w and 0 <= pred_v <= orig_h:
                    final_u, final_v = pred_u, pred_v
                    last_u, last_v = pred_u, pred_v
                    missed_frames += 1

            if final_u:
                scaled_u = final_u * scale_x
                scaled_v = final_v * scale_y
                self.trajectory.append({'frame': frame_id, 'u': scaled_u, 'v': scaled_v, 'type': 'real' if best_candidate else 'predicted'})

            if out:
                if final_u:
                    # Green = Real detection, Red = Prediction
                    colore = (0, 255, 0) if best_candidate else (0, 0, 255)
                    cv2.circle(frame, (int(final_u), int(final_v)), 6, colore, -1)
                out.write(frame)

            frame_id += 1
            if frame_id % 50 == 0: print(f"   ... frame {frame_id}")

        cap.release()
        if out: out.release()

        self.trajectory = self.pulisci_traiettoria(self.trajectory)
        print(f"✅ Finished. {len(self.trajectory)} points tracked.")
        return self.trajectory

In [6]:
# CELLA 3: ESECUZIONE
lista_config = [
    # (Nome File Tuo, Larghezza Target Membro 1, Altezza Target Membro 1)
    ("Cemento_Tennis.mp4", 1075, 613),
    ("Erba_Tennis1.mp4", 1068, 609),
    ("Terra_Tennis1.mp4", 1069, 601)
]

print(f"🚀 AVVIO ANALISI HD + EXPORT COMPATIBILE...\n")
membro2 = IlSegugio()

for nome_video, t_w, t_h in lista_config:
    if not os.path.exists(nome_video):
        print(f"⚠️ Video {nome_video} non trovato. Caricalo!")
        continue

    print(f"🎥 Elaborazione: {nome_video}...")
    path_video_output = f"OUTPUT_{nome_video}"
    path_dati_output = f"DATI_{nome_video.replace('.mp4', '.json')}"

    # Passiamo le dimensioni target alla funzione
    dati = membro2.processa_video(nome_video, t_w, t_h, video_output=path_video_output)

    with open(path_dati_output, 'w') as f:
        json.dump(dati, f, indent=4)

    try:
        files.download(path_video_output)
        files.download(path_dati_output)
    except:
        pass
    print("-" * 50)

🚀 AVVIO ANALISI HD + EXPORT COMPATIBILE...

🐶 Initializing 'Il Segugio' (Stabilized & Enhanced)...
🎥 Elaborazione: Cemento_Tennis.mp4...
▶️ Processing: Cemento_Tennis.mp4 (2880x1800)
   ... frame 50
   ... frame 100
   ... frame 150
   ... frame 200
✅ Finished. 246 points tracked.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

--------------------------------------------------
